# 2. Neural Style Transfer on your local machine

To make sure all the scripts that are working correctly, we need to test them on our local machine first. Specifically, we'll be testing the components that will be in our __aci app__ and our __aks app__.

The __aci app__ is responsible for:
1. preprocessing the images into individual frames and extracting the audio file
2. adding references of the individual images to the Service Bus queue
3. waiting until style transfer has been applied to all images
4. postprocessing the style transfer images by stitching them together into a video and re-adding the audio clip

The __aks app__ is responsible for:
1. polling the Service Bus queue for new images 
2. applying style transfer to those images and saving them back to storage

---

### Import package and load .env

In [1]:
from dotenv import set_key, get_key, find_dotenv, load_dotenv
from pathlib import Path
import json
import os

In [2]:
env_path = find_dotenv(raise_error_if_not_found=True)
load_dotenv(env_path)

True

### Preprocess video 
The first thing we want to test locally is the preprocessing step. This step will use a reference to the video file in blob and perform the following:
1. Download the video to a tmp location
2. Use `ffmpeg` to extract the audio file from the video and save the file to a tmp location
3. Use `ffmpeg` to split the video up into individual frames and save all frames to the tmp location
4. Upload the extracted audio file & frames to storage to be consumed by the next step

In [3]:
input_frames_dir = "orangutan_frames_test"
audio_file = "orangutan_audio_test.aac"

Run the preprocess script:
- `--frames-dir` is the directory in Azure blob that will be created to store individual frames from preprocessing the video.
- `--audio` is the audio file name in Azure blob to save the extracted audio clip to.

After running this step, check your storage account to see if your `frames_dir` and `audio_file` is loaded.

In [4]:
!python aci/preprocess.py \
    --video orangutan.mp4 \
    --frames-dir {input_frames_dir} \
    --audio {audio_file}

ffmpeg version 4.0.2 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 4.8.2 (GCC) 20140120 (Red Hat 4.8.2-15)
  configuration: --prefix=/home/jiata/anaconda3/envs/batchscoringdl --disable-doc --disable-openssl --enable-shared --enable-static --extra-cflags='-Wall -g -m64 -pipe -O3 -march=x86-64 -fPIC' --extra-cxxflags='-Wall -g -m64 -pipe -O3 -march=x86-64 -fPIC' --extra-libs='-lpthread -lm -lz' --enable-zlib --enable-pic --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --enable-libfreetype --enable-gnutls --enable-libx264 --enable-libopenh264
  libavutil      56. 14.100 / 56. 14.100
  libavcodec     58. 18.100 / 58. 18.100
  libavformat    58. 12.100 / 58. 12.100
  libavdevice    58.  3.100 / 58.  3.100
  libavfilter     7. 16.100 /  7. 16.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  1.100 /  5.  1.100
  libswresample   3.  1.100 /  3.  1.100
  libpostproc    55.  1.100 / 55.  1.100
Input #0, mov,mp4,m4a

### Test neural style transfer locally

Now that the video is preprocessed into individual frames, test the style transfer script locally:
- Add 10 images to queue
- Dequeue those 10 images and apply neural style transfer locally
- Check that the queue is now empty

In [5]:
output_frames_dir = "orangutan_frames_local_test_processed"

In [6]:
!python aci/add_images_to_queue.py \
    --input-dir {input_frames_dir} \
    --output-dir {output_frames_dir} \
    --queue-limit 10

Adding images from 'orangutan_frames_test' in storage to queue 'batchscoringdlqueue'
2018-12-07 20:36:02,364 [root:add_images_to_queue.py:48] DEBUG - Queue limit is reached. Exiting process...


The `--input-dir` is the directory in your blob container that corresponds to the queue messages (the script will look inside the directory specified by `--input-dir` to find and process the message that it dequeued).

In [7]:
!python aks/main.py --terminate

2018-12-07 20:36:03,072 [root:process_images_from_queue.py:35] DEBUG - Downloading style model from directory models
2018-12-07 20:36:03,740 [root:process_images_from_queue.py:50] DEBUG - The following model were downloaded: ["candy.pth","model.pth","mosaic.pth","rain_princess.pth","udnie.pth"]
2018-12-07 20:36:03,740 [root:process_images_from_queue.py:88] DEBUG - It took 0.70 seconds to download style model.
2018-12-07 20:36:03,740 [root:process_images_from_queue.py:91] DEBUG - Start listening to queue 'batchscoringdlqueue' on service bus...
2018-12-07 20:36:03,740 [root:process_images_from_queue.py:96] DEBUG - Peek queue...
2018-12-07 20:36:03,759 [root:process_images_from_queue.py:123] DEBUG - Queue message body: {'input_frame': '000001_frame.jpg', 'input_dir': 'orangutan_frames_test', 'output_dir': 'orangutan_frames_local_test_processed'}
2018-12-07 20:36:03,767 [root:process_images_from_queue.py:140] DEBUG - Starting style transfer on orangutan_frames_test/000001_frame.jpg
2018-12

2018-12-07 20:36:09,098 [root:style_transfer.py:157] DEBUG - Processing .aks/input/000009_frame.jpg
2018-12-07 20:36:09,180 [root:process_images_from_queue.py:150] DEBUG - Finished style transfer on orangutan_frames_test/000009_frame.jpg
2018-12-07 20:36:09,204 [root:process_images_from_queue.py:162] DEBUG - Uploaded output file and log file to storage
2018-12-07 20:36:09,204 [root:process_images_from_queue.py:171] DEBUG - Deleting queue message...
2018-12-07 20:36:09,304 [root:process_images_from_queue.py:96] DEBUG - Peek queue...
2018-12-07 20:36:09,310 [root:process_images_from_queue.py:123] DEBUG - Queue message body: {'input_frame': '000010_frame.jpg', 'input_dir': 'orangutan_frames_test', 'output_dir': 'orangutan_frames_local_test_processed'}
2018-12-07 20:36:09,356 [root:process_images_from_queue.py:140] DEBUG - Starting style transfer on orangutan_frames_test/000010_frame.jpg
2018-12-07 20:36:09,383 [root:style_transfer.py:157] DEBUG - Processing .aks/input/000010_frame.jpg
201

In [8]:
!az servicebus queue show \
    --name {get_key(env_path, "SB_QUEUE")} \
    --namespace-name {get_key(env_path, "SB_NAMESPACE")} \
    --resource-group {get_key(env_path, "RESOURCE_GROUP")} \
    --query "countDetails.activeMessageCount"

0


---

Continue to the next [notebook](/notebooks/03_style_transfer_on_aks.ipynb).